<a href="https://colab.research.google.com/github/Mmabatho/Week6_AI_For_Software_Engneering/blob/chris/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Edge AI Prototype: Recyclable Items Classification
# Task 1: Train a lightweight model and convert to TensorFlow Lite

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import os

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

class RecyclableClassifier:
    def __init__(self, num_classes=4):
        """
        Initialize the recyclable items classifier
        Classes: 0-Plastic, 1-Paper, 2-Metal, 3-Glass
        """
        self.num_classes = num_classes
        self.class_names = ['Plastic', 'Paper', 'Metal', 'Glass']
        self.model = None
        self.tflite_model = None

    def create_model(self, input_shape=(224, 224, 3)):
        """Create a lightweight model based on MobileNetV2"""
        # Load pre-trained MobileNetV2 as base model
        base_model = MobileNetV2(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )

        # Freeze base model layers
        base_model.trainable = False

        # Add custom classification head
        inputs = tf.keras.Input(shape=input_shape)
        x = base_model(inputs, training=False)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.2)(x)
        outputs = Dense(self.num_classes, activation='softmax')(x)

        self.model = Model(inputs, outputs)

        # Compile the model
        self.model.compile(
            optimizer=Adam(learning_rate=0.0001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return self.model

    def generate_synthetic_data(self, samples_per_class=100):
        """Generate synthetic data for demonstration purposes"""
        print("Generating synthetic training data...")

        # Create synthetic images with different patterns for each class
        X_train = []
        y_train = []
        X_test = []
        y_test = []

        for class_idx in range(self.num_classes):
            # Training data
            for _ in range(samples_per_class):
                # Generate synthetic image with class-specific patterns
                img = np.random.rand(224, 224, 3)

                # Add class-specific patterns
                if class_idx == 0:  # Plastic - add blue tint
                    img[:, :, 2] += 0.3
                elif class_idx == 1:  # Paper - add brown/yellow tint
                    img[:, :, 0] += 0.2
                    img[:, :, 1] += 0.2
                elif class_idx == 2:  # Metal - add silver/gray tint
                    img[:, :, :] += 0.1
                elif class_idx == 3:  # Glass - add transparency effect
                    img = img * 0.8 + 0.2

                img = np.clip(img, 0, 1)
                X_train.append(img)
                y_train.append(class_idx)

            # Test data (20% of training size)
            for _ in range(samples_per_class // 5):
                img = np.random.rand(224, 224, 3)

                # Same class-specific patterns
                if class_idx == 0:
                    img[:, :, 2] += 0.3
                elif class_idx == 1:
                    img[:, :, 0] += 0.2
                    img[:, :, 1] += 0.2
                elif class_idx == 2:
                    img[:, :, :] += 0.1
                elif class_idx == 3:
                    img = img * 0.8 + 0.2

                img = np.clip(img, 0, 1)
                X_test.append(img)
                y_test.append(class_idx)

        # Convert to numpy arrays and shuffle
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        # Shuffle the data
        train_indices = np.random.permutation(len(X_train))
        test_indices = np.random.permutation(len(X_test))

        X_train = X_train[train_indices]
        y_train = y_train[train_indices]
        X_test = X_test[test_indices]
        y_test = y_test[test_indices]

        print(f"Training data shape: {X_train.shape}")
        print(f"Test data shape: {X_test.shape}")

        return X_train, y_train, X_test, y_test

    def train_model(self, X_train, y_train, X_val, y_val, epochs=10):
        """Train the model"""
        print("Training the model...")

        # Define callbacks
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=3,
            restore_best_weights=True
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=2,
            min_lr=0.00001
        )

        # Train the model
        history = self.model.fit(
            X_train, y_train,
            batch_size=32,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )

        return history

    def evaluate_model(self, X_test, y_test):
        """Evaluate model performance"""
        print("Evaluating model performance...")

        # Get predictions
        predictions = self.model.predict(X_test)
        y_pred = np.argmax(predictions, axis=1)

        # Calculate accuracy
        accuracy = np.mean(y_pred == y_test)

        # Print classification report
        print(f"\nTest Accuracy: {accuracy:.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred, target_names=self.class_names))

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("\nConfusion Matrix:")
        print(cm)

        return accuracy, y_pred

    def convert_to_tflite(self):
        """Convert the trained model to TensorFlow Lite format"""
        print("Converting model to TensorFlow Lite...")

        # Convert to TensorFlow Lite
        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)

        # Enable optimizations for edge deployment
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

        # Convert the model
        self.tflite_model = converter.convert()

        # Save the TFLite model
        with open('recyclable_classifier.tflite', 'wb') as f:
            f.write(self.tflite_model)

        # Get model sizes
        original_size = os.path.getsize('recyclable_classifier.tflite') / 1024  # KB
        print(f"TensorFlow Lite model size: {original_size:.2f} KB")

        return self.tflite_model

    def test_tflite_model(self, X_test, y_test, num_samples=10):
        """Test the TensorFlow Lite model"""
        print("Testing TensorFlow Lite model...")

        # Load TFLite model and allocate tensors
        interpreter = tf.lite.Interpreter(model_content=self.tflite_model)
        interpreter.allocate_tensors()

        # Get input and output tensors
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Test on a subset of test data
        correct_predictions = 0

        for i in range(min(num_samples, len(X_test))):
            # Prepare input data
            input_data = np.expand_dims(X_test[i].astype(np.float32), axis=0)

            # Set input tensor
            interpreter.set_tensor(input_details[0]['index'], input_data)

            # Run inference
            interpreter.invoke()

            # Get output
            output_data = interpreter.get_tensor(output_details[0]['index'])
            prediction = np.argmax(output_data[0])

            if prediction == y_test[i]:
                correct_predictions += 1

            print(f"Sample {i+1}: True={self.class_names[y_test[i]]}, "
                  f"Predicted={self.class_names[prediction]}")

        tflite_accuracy = correct_predictions / num_samples
        print(f"\nTensorFlow Lite Model Accuracy: {tflite_accuracy:.4f}")

        return tflite_accuracy

def main():
    """Main execution function"""
    print("=== Edge AI Prototype: Recyclable Items Classification ===\n")

    # Initialize classifier
    classifier = RecyclableClassifier()

    # Create the model
    model = classifier.create_model()
    print("Model architecture:")
    model.summary()

    # Generate synthetic data
    X_train, y_train, X_test, y_test = classifier.generate_synthetic_data(
        samples_per_class=200
    )

    # Train the model
    history = classifier.train_model(X_train, y_train, X_test, y_test, epochs=15)

    # Evaluate the model
    accuracy, predictions = classifier.evaluate_model(X_test, y_test)

    # Convert to TensorFlow Lite
    tflite_model = classifier.convert_to_tflite()

    # Test TensorFlow Lite model
    tflite_accuracy = classifier.test_tflite_model(X_test, y_test, num_samples=20)

    # Print summary
    print("\n" + "="*50)
    print("DEPLOYMENT SUMMARY")
    print("="*50)
    print(f"Original Model Accuracy: {accuracy:.4f}")
    print(f"TensorFlow Lite Model Accuracy: {tflite_accuracy:.4f}")
    print(f"Model file: recyclable_classifier.tflite")
    print("\nEdge AI Benefits:")
    print("1. Real-time inference without internet connection")
    print("2. Reduced latency (no cloud communication)")
    print("3. Enhanced privacy (data stays on device)")
    print("4. Lower bandwidth usage")
    print("5. Reliable operation in remote areas")

if __name__ == "__main__":
    main()

# Additional utility functions for Edge AI deployment

def simulate_edge_deployment():
    """Simulate deployment on edge device (like Raspberry Pi)"""
    print("\n=== Simulating Edge Device Deployment ===")

    # Simulate loading model on edge device
    print("Loading TensorFlow Lite model on edge device...")

    # Simulate real-time inference
    print("Simulating real-time camera input...")

    # Mock camera input
    camera_input = np.random.rand(224, 224, 3)

    # Load and run inference
    interpreter = tf.lite.Interpreter(model_path='recyclable_classifier.tflite')
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Preprocess input
    input_data = np.expand_dims(camera_input.astype(np.float32), axis=0)

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Get results
    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = np.argmax(output_data[0])
    confidence = np.max(output_data[0])

    class_names = ['Plastic', 'Paper', 'Metal', 'Glass']
    print(f"Detected: {class_names[prediction]} (Confidence: {confidence:.2f})")
    print("Processing time: ~50ms (typical for edge device)")

# Run the main program
if __name__ == "__main__":
    main()
    simulate_edge_deployment()

=== Edge AI Prototype: Recyclable Items Classification ===

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Model architecture:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         5,124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,263,108 (8.63 MB)

 Trainable params: 5,124 (20.02 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         5,124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,263,108 (8.63 MB)

 Trainable params: 5,124 (20.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Generating synthetic training data...
Training data shape: (800, 224, 224, 3)
Test data shape: (160, 224, 224, 3)
Training the model...
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.2515 - loss: 2.3387 - val_accuracy: 0.2562 - val_loss: 1.8037 - learning_rate: 1.0000e-04
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 793ms/step - accuracy: 0.2358 - loss: 1.8756 - val_accuracy: 0.2562 - val_loss: 1.5245 - learning_rate: 1.0000e-04
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 879ms/step - accuracy: 0.2508 - loss: 1.6050 - val_accuracy: 0.2750 - val_loss: 1.4347 - learning_rate: 1.0000e-04
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 877ms/step - accuracy: 0.2262 - loss: 1.5805 - val_accuracy: 0.2688 - val_loss: 1.3884 - learning_rate: 1.0000e-04
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 815ms/step - accuracy: 0.2524 - loss: 1.5507 - val_accuracy: 0.3250 - val_loss: 1.3458 - learning_rate: 1.0000e-04
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 859ms/step - accuracy: 0.2645 - loss: 1.